In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from datetime import datetime

# Get today's date
end_date = datetime.now().strftime('%Y-%m-%d')

# Download Bitcoin data
btc_data = yf.download('BTC-USD', start='2023-01-01', end=end_date)

# Use only the 'Close' prices
btc_data = btc_data['Close']

# Set frequency to daily
btc_data = btc_data.asfreq('D')

# Split data into training and test sets
train_size = int(len(btc_data) * 0.8)
train, test = btc_data[0:train_size], btc_data[train_size:]

# Find the best ARIMA parameters
best_mse = float("inf")
best_order = None

for p in range(6):  # p=0,1,2,3,4,5
    for d in range(3):  # d=0,1,2
        for q in range(6):  # q=0,1,2,3,4,5
            try:
                model = ARIMA(train, order=(p, d, q))
                model_fit = model.fit()
                predictions = model_fit.forecast(steps=len(test))
                
                mse = mean_squared_error(test, predictions)
                if mse < best_mse:
                    best_mse = mse
                    best_order = (p, d, q)
            except:
                continue

# Print the best parameters and MSE
print(f'Best ARIMA parameters: p={best_order[0]}, d={best_order[1]}, q={best_order[2]} with MSE={best_mse:.2f}')

# Fit the best model and forecast
best_model = ARIMA(train, order=best_order)
best_model_fit = best_model.fit()
forecast = best_model_fit.forecast(steps=7)

# Last closing price
last_close = btc_data.iloc[-1]

# Store the last close price and predicted prices in a DataFrame
predicted_prices = pd.DataFrame({
    'Date': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7),
    'Predicted Price': forecast
})

# Set the first prediction as the last closing price
predicted_prices.loc[0, 'Predicted Price'] = last_close  # First prediction is the last close price
predicted_prices['Predicted Price'] = predicted_prices['Predicted Price'].shift(-1)  # Shift predictions

# Drop NaN values
predicted_prices = predicted_prices.dropna()

# Calculate MSE and MAE
mse = mean_squared_error(test[-len(predicted_prices):], predicted_prices['Predicted Price'])
mae = mean_absolute_error(test[-len(predicted_prices):], predicted_prices['Predicted Price'])

print(predicted_prices)
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')

# Visualize results
plt.figure(figsize=(12, 6))
plt.plot(btc_data.index, btc_data, label='Actual Price', color='blue')
plt.plot(predicted_prices['Date'], predicted_prices['Predicted Price'], label='Predicted Price', color='orange', marker='o')
plt.axvline(x=test.index[-1], color='red', linestyle='--', label='Forecast Start')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid()
plt.show()


ModuleNotFoundError: No module named 'statsmodels'